In [1]:
%run init.ipynb
import nltk
nltk.download('punkt')

matchzoo version 1.1.1
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
/home/ec2-user/.matchzoo/datasets/wiki_qa/WikiQACorpus
/home/ec2-user/.matchzoo/datasets/wiki_qa/WikiQACorpus
/home/ec2-user/.matchzoo/datasets/wiki_qa/WikiQACorpus
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
preprocessor = mz.preprocessors.BasicPreprocessor(
    truncated_length_left = 10,
    truncated_length_right = 40,
    filter_low_freq = 2
)

In [4]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7938.96it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4197.29it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404432/404432 [00:00<00:00, 987748.43it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7695.93it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4298.11it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 481175.41it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 8027.50it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00:

In [5]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f438ac5c8d0>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f4409cc6d68>,
 'vocab_size': 16675,
 'embedding_input_dim': 16675}

In [6]:
preprocessor.context['vocab_unit'].context

{'term_index': {'<PAD>': 0,
  '<OOV>': 1,
  '0': 2,
  '007': 3,
  '1': 4,
  '10': 5,
  '100': 6,
  '1000': 7,
  '1001': 8,
  '100th': 9,
  '101': 10,
  '102': 11,
  '103': 12,
  '1040': 13,
  '1040a': 14,
  '105': 15,
  '1066': 16,
  '108': 17,
  '109': 18,
  '10th': 19,
  '11': 20,
  '110': 21,
  '111': 22,
  '112th': 23,
  '113': 24,
  '114': 25,
  '117': 26,
  '118': 27,
  '11th': 28,
  '12': 29,
  '120': 30,
  '1200': 31,
  '121': 32,
  '1213': 33,
  '123': 34,
  '124': 35,
  '125': 36,
  '1279': 37,
  '12th': 38,
  '13': 39,
  '130': 40,
  '134': 41,
  '1350': 42,
  '1394': 43,
  '13th': 44,
  '14': 45,
  '140': 46,
  '141': 47,
  '145': 48,
  '1492': 49,
  '14th': 50,
  '15': 51,
  '150': 52,
  '1500': 53,
  '1506': 54,
  '1521': 55,
  '1532': 56,
  '1534': 57,
  '1535': 58,
  '1547': 59,
  '1570': 60,
  '1588': 61,
  '1595': 62,
  '15th': 63,
  '16': 64,
  '160': 65,
  '1600': 66,
  '1601': 67,
  '1603': 68,
  '1607': 69,
  '1619': 70,
  '1620': 71,
  '1631': 72,
  '1642': 73,
 

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [8]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=1
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)

In [14]:
trainset.batch_size

32

In [10]:
padding_callback = mz.models.KNRM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    #batch_size=20,
    stage='train',
    #resample=True,
    #sort=False,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    #batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [11]:
model = mz.models.KNRM()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['kernel_num'] = 21
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

KNRM(
  (embedding): Embedding(16675, 100, padding_idx=0)
  (kernels): ModuleList(
    (0): GaussianKernel()
    (1): GaussianKernel()
    (2): GaussianKernel()
    (3): GaussianKernel()
    (4): GaussianKernel()
    (5): GaussianKernel()
    (6): GaussianKernel()
    (7): GaussianKernel()
    (8): GaussianKernel()
    (9): GaussianKernel()
    (10): GaussianKernel()
    (11): GaussianKernel()
    (12): GaussianKernel()
    (13): GaussianKernel()
    (14): GaussianKernel()
    (15): GaussianKernel()
    (16): GaussianKernel()
    (17): GaussianKernel()
    (18): GaussianKernel()
    (19): GaussianKernel()
    (20): GaussianKernel()
  )
  (out): Linear(in_features=21, out_features=1, bias=True)
)
Trainable params:  1667522


In [12]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [15]:
#rankhingeloss
trainer.run()

[Iter-480 Loss-0.019]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5307 - normalized_discounted_cumulative_gain@5(0.0): 0.5809 - mean_average_precision(0.0): 0.5491



KeyboardInterrupt: 

In [16]:
torch.cuda.is_available()
torch.tensor([1.0, 2.0]).cuda()

RuntimeError: CUDA error: no kernel image is available for execution on the device

In [18]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [19]:
torch.__version__

'1.6.0'

In [10]:
import torch.nn.functional as F
F.softmax(input, dim = 1)

tensor([[0.0542, 0.0470, 0.5194, 0.1771, 0.2024],
        [0.1723, 0.0305, 0.1617, 0.0800, 0.5555],
        [0.1479, 0.0760, 0.3140, 0.1299, 0.3323],
        [0.0850, 0.2024, 0.6358, 0.0147, 0.0621]])

In [24]:
dev_pack_processed.left.loc["Q11"].text_left

[7484, 2040, 8196, 1, 14008, 7732, 7482, 15653]

In [32]:
for (inputs, target) in trainer._trainloader:
    print(inputs)

{'text_left': tensor([[    0,     0, 16367, 16257,  3246,  2914, 12385,  6198, 10106],
        [    0,     0, 16367, 16257,  3246,  2914, 12385,  6198, 10106],
        [    0, 16367,  4898,     4, 16605,  9457, 15275, 15108, 11429],
        [    0, 16367,  4898,     4, 16605,  9457, 15275, 15108, 11429],
        [    0,     0, 16367,  4898,   738, 10079, 15652,  2858,  9457],
        [    0,     0, 16367,  4898,   738, 10079, 15652,  2858,  9457],
        [    0,     0,     0, 16374, 15108, 16443,  2156,  8267, 11078],
        [    0,     0,     0, 16374, 15108, 16443,  2156,  8267, 11078],
        [    0,     0,     0,     0, 16376,  8196,  9083,  6216,  1542],
        [    0,     0,     0,     0, 16376,  8196,  9083,  6216,  1542],
        [    0,  7484,  1362,  3034,  1138, 15662, 15461,  1138, 11809],
        [    0,  7484,  1362,  3034,  1138, 15662, 15461,  1138, 11809],
        [    0,     0,     0,     0,     0,     0, 16397,  6404, 16217],
        [    0,     0,     0,     0, 

{'text_left': tensor([[    0,     0,     0,     0,     0, 16367,  8196,  1648, 12189, 10941],
        [    0,     0,     0,     0,     0, 16367,  8196,  1648, 12189, 10941],
        [16397,  4755, 15108,   322,  9915, 10561,  9341, 11294,  1138,  4390],
        [16397,  4755, 15108,   322,  9915, 10561,  9341, 11294,  1138,  4390],
        [    0,     0, 16397, 16257, 15108,  6198, 10618, 15275,  8128,  9495],
        [    0,     0, 16397, 16257, 15108,  6198, 10618, 15275,  8128,  9495],
        [    0,     0, 16367, 10998, 10561,  1919,  1362, 13103,  4145,  6500],
        [    0,     0, 16367, 10998, 10561,  1919,  1362, 13103,  4145,  6500],
        [    0,     0,     0,     0,     0,     0, 16367,  1362, 15108, 14639],
        [    0,     0,     0,     0,     0,     0, 16367,  1362, 15108, 14639],
        [    0,     0,     0,     0,     0, 16376,  1362,  9186, 15391,  9156],
        [    0,     0,     0,     0,     0, 16376,  1362,  9186, 15391,  9156],
        [    0,     0,    

{'text_left': tensor([[    0,     0,     0,     0,     0,     0, 16367,  1362,   738, 10628],
        [    0,     0,     0,     0,     0,     0, 16367,  1362,   738, 10628],
        [    0,     0,     0,     0, 16397,  8196, 10616, 15108,     5,  2050],
        [    0,     0,     0,     0, 16397,  8196, 10616, 15108,     5,  2050],
        [    0,     0,     0, 16374,  4898,  2114,  6491, 14329,  6336,  3054],
        [    0,     0,     0, 16374,  4898,  2114,  6491, 14329,  6336,  3054],
        [    0,     0,     0,  7484,  9289, 11171,  8196,   548,  1844, 11512],
        [    0,     0,     0,  7484,  9289, 11171,  8196,   548,  1844, 11512],
        [    0,     0,     0,     0,     0, 16367,  4656,  1976,  1404,  4657],
        [    0,     0,     0,     0,     0, 16367,  4656,  1976,  1404,  4657],
        [    0,     0,     0, 16374,  4656, 13578,  3360, 10616, 15108,  9323],
        [    0,     0,     0, 16374,  4656, 13578,  3360, 10616, 15108,  9323],
        [    0,     0,    

{'text_left': tensor([[    0,     0,     0,     0,     0,  7484,  9289, 11089,  7732,  2994],
        [    0,     0,     0,     0,     0,  7484,  9289, 11089,  7732,  2994],
        [    0, 16374, 16257,  8267,  9158,  7732, 15108,  7476, 10561, 12730],
        [    0, 16374, 16257,  8267,  9158,  7732, 15108,  7476, 10561, 12730],
        [    0,     0, 16374, 16257, 15108,  6198,  9357, 11575,  2211, 16555],
        [    0,     0, 16374, 16257, 15108,  6198,  9357, 11575,  2211, 16555],
        [    0,     0,     0,  7484,  9289,  1079,  7732, 15108, 15873,  3683],
        [    0,     0,     0,  7484,  9289,  1079,  7732, 15108, 15873,  3683],
        [    0,     0,     0,  7484,  4881,  6358,  1611,  5702,  6160, 12724],
        [    0,     0,     0,  7484,  4881,  6358,  1611,  5702,  6160, 12724],
        [    0,     0,     0,     0, 16367,  4656,  8714,  8371,  2901,  4881],
        [    0,     0,     0,     0, 16367,  4656,  8714,  8371,  2901,  4881],
        [    0,     0,    

{'text_left': tensor([[    0, 16367,  7101, 15275, 15108,  8876,  7799, 12371, 10561, 11313],
        [    0, 16367,  7101, 15275, 15108,  8876,  7799, 12371, 10561, 11313],
        [    0, 16367,  8196,  2672,    45,  4231,  8196,   548, 15660, 10561],
        [    0, 16367,  8196,  2672,    45,  4231,  8196,   548, 15660, 10561],
        [    0,     0,     0,     0,  7484, 16257,  3331,  8115,  7732,  6145],
        [    0,     0,     0,     0,  7484, 16257,  3331,  8115,  7732,  6145],
        [    0,     0,     0,     0,     0,     0, 16367,  8196, 13832,  8657],
        [    0,     0,     0,     0,     0,     0, 16367,  8196, 13832,  8657],
        [    0,     0,     0,     0,     0, 16367,  8196,  8202, 10675,  7858],
        [    0,     0,     0,     0,     0, 16367,  8196,  8202, 10675,  7858],
        [    0,     0,     0, 16367,  8196, 15108, 13798, 10561,  6818,  6491],
        [    0,     0,     0, 16367,  8196, 15108, 13798, 10561,  6818,  6491],
        [    0,     0,    

...

{'text_left': tensor([[    0,  7484,  9029,  2603, 16626,  1887,  7732, 15108, 14717,  3937],
        [    0,  7484,  9029,  2603, 16626,  1887,  7732, 15108, 14717,  3937],
        [    0,     0,     0,     0,     0, 16374,  4881, 14012,  5149,  7099],
        [    0,     0,     0,     0,     0, 16374,  4881, 14012,  5149,  7099],
        [    0,     0,     0,     0,     0, 16367,  3492, 15885, 14077,  9367],
        [    0,     0,     0,     0,     0, 16367,  3492, 15885, 14077,  9367],
        [    0,     0,     0, 16376,  4656,  9320,  8253, 11459, 11867,  1846],
        [    0,     0,     0, 16376,  4656,  9320,  8253, 11459, 11867,  1846],
        [    0,     0,     0,     0,     0,     0, 16367,  1362,  8984, 10736],
        [    0,     0,     0,     0,     0,     0, 16367,  1362,  8984, 10736],
        [12650,  8196,  8231, 15275,  1479, 14042, 15275,  6522, 15141, 11954],
        [12650,  8196,  8231, 15275,  1479, 14042, 15275,  6522, 15141, 11954],
        [    0,     0,    

{'text_left': tensor([[    0, 16367,  8196, 15108, 10702,  8722,  7732,  1110,  1093, 13371],
        [    0, 16367,  8196, 15108, 10702,  8722,  7732,  1110,  1093, 13371],
        [    0,     0,     0,     0,  7484, 16257,  3331,  8115,  7732,  6145],
        [    0,     0,     0,     0,  7484, 16257,  3331,  8115,  7732,  6145],
        [    0,     0, 16374, 16257, 12137,  7161,  2514,  8327,  7248,  9156],
        [    0,     0, 16374, 16257, 12137,  7161,  2514,  8327,  7248,  9156],
        [    0,     0,     0,     0,     0, 16376,  4656,  5569,  3360,  6500],
        [    0,     0,     0,     0,     0, 16376,  4656,  5569,  3360,  6500],
        [    0,     0,     0,     0, 16367, 16605,  9045, 10561, 12984,  9156],
        [    0,     0,     0,     0, 16367, 16605,  9045, 10561, 12984,  9156],
        [    0,     0,     0, 16367,  8658,  1362, 14205,  7732, 14084,   866],
        [    0,     0,     0, 16367,  8658,  1362, 14205,  7732, 14084,   866],
        [    0,     0,    

{'text_left': tensor([[    0,     0,     0,     0,     0, 16367,  1362, 15108,   497,  3733],
        [    0,     0,     0,     0,     0, 16367,  1362, 15108,   497,  3733],
        [    0,     0,     0,     0,     0,     0, 16367,  3331,  8196,  2476],
        [    0,     0,     0,     0,     0,     0, 16367,  3331,  8196,  2476],
        [    0, 16367,  4898, 15108,  6918,  9457, 10616, 15108,  9627,  6217],
        [    0, 16367,  4898, 15108,  6918,  9457, 10616, 15108,  9627,  6217],
        [    0,     0,     0,     0,     0, 16374, 16257, 16260,  5248, 11784],
        [    0,     0,     0,     0,     0, 16374, 16257, 16260,  5248, 11784],
        [    0,     0,     0, 16367,  8196, 15108, 11921, 10561, 14425,  2615],
        [    0,     0,     0, 16367,  8196, 15108, 11921, 10561, 14425,  2615],
        [    0,     0,     0,     0, 16367,  8196,  2114,  7049, 15302,  4769],
        [    0,     0,     0,     0, 16367,  8196,  2114,  7049, 15302,  4769],
        [    0,     0,    

{'text_left': tensor([[    0,     0,     0,     0,  7484,  1362,   926, 12248,  5450,  2450],
        [    0,     0,     0,     0,  7484,  1362,   926, 12248,  5450,  2450],
        [    0,     0,     0,  7484,  9289, 11160,  8967,  7732,  9544, 15035],
        [    0,     0,     0,  7484,  9289, 11160,  8967,  7732,  9544, 15035],
        [    0,     0,     0,     0, 16376,  1362, 11519,  4222,  6498, 13480],
        [    0,     0,     0,     0, 16376,  1362, 11519,  4222,  6498, 13480],
        [    0,     0,     0,     0, 16367,  3921,  1362,  7732,  3625, 14718],
        [    0,     0,     0,     0, 16367,  3921,  1362,  7732,  3625, 14718],
        [    0,     0,     0,     0,     0, 16397,  8196, 15108,  6957,  5460],
        [    0,     0,     0,     0,     0, 16397,  8196, 15108,  6957,  5460],
        [    0,     0,     0,     0,     0,     0, 16367,  4881, 15670,  4579],
        [    0,     0,     0,     0,     0,     0, 16367,  4881, 15670,  4579],
        [    0,     0,    

{'text_left': tensor([[    0,     0,     0,     0, 16367, 15275,  9211, 16474,  8912],
        [    0,     0,     0,     0, 16367, 15275,  9211, 16474,  8912],
        [    0,     0,     0,     0, 16367,  8196,  6943,  9697, 14426],
        [    0,     0,     0,     0, 16367,  8196,  6943,  9697, 14426],
        [    0,     0,     0,     0,  7484,  4898,   548,  5006, 16514],
        [    0,     0,     0,     0,  7484,  4898,   548,  5006, 16514],
        [    0,     0,     0,     0, 16367, 15275,  9211, 16474,  8912],
        [    0,     0,     0,     0, 16367, 15275,  9211, 16474,  8912],
        [    0,     0, 16367,  3921,  1362, 15727, 15108,  2432, 12628],
        [    0,     0, 16367,  3921,  1362, 15727, 15108,  2432, 12628],
        [    0,     0, 16367,  8196,  7732, 15108,  1036,  9978, 14849],
        [    0,     0, 16367,  8196,  7732, 15108,  1036,  9978, 14849],
        [    0,     0,     0, 16376,  1362,  6737, 10937,  1896,  6401],
        [    0,     0,     0, 16376, 

In [35]:
trainer._trainloader.label

/Users/yyang/opt/anaconda3/envs/conda_env/lib/python3.7/site-packages/matchzoo_py-1.1.1-py3.7.egg/matchzoo/data_pack/data_pack.py:151: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x[key] = np.array(val)


array([1., 0., 1., ..., 0., 1., 0.])